In [1]:
# Parameters
DB_PATH = "C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-92\\test_notebooks_execute_quickst0\\interactions.db"
SEARCH_LEDGER_PATH = "C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-92\\test_notebooks_execute_quickst0\\search_telemetry.json"


# DataLab Quickstart
This notebook demonstrates how to connect to the shared Playground data store (SQLite, Cosmos DB, or JSON depending on configuration),
normalize the typing metadata JSON, and compute lightweight aggregates.

In [2]:
from __future__ import annotations

import json
import os
from pathlib import Path
import sys

import pandas as pd

ROOT = Path.cwd().parents[1]
os.environ.setdefault("LAB_ROOT", str(ROOT))
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from kitchen.scripts.metrics import load_interactions

DB_OVERRIDE = globals().get("DB_PATH") or os.environ.get("DATALAB_DB_PATH")
INTERACTION_LIMIT = int(os.environ.get("DATALAB_INTERACTION_LIMIT", "1000"))
metrics_args = {"limit": INTERACTION_LIMIT}
if DB_OVERRIDE:
    metrics_args = {"db_path": Path(DB_OVERRIDE).expanduser().resolve()}
DATA_SOURCE = str(metrics_args.get("db_path", "data store"))
df = load_interactions(**metrics_args)
DATA_SOURCE

'C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-92\\test_notebooks_execute_quickst0\\interactions.db'

In [3]:
df.head()

,id,user_prompt_text,typing_metadata_json,created_at
0,prompt-002,Follow-up,"{""total_duration_ms"": 900, ""keystroke_events"":...",2025-01-02T00:00:00Z
1,prompt-001,Quickstart prompt,"{""total_duration_ms"": 1500, ""keystroke_events""...",2025-01-01T00:00:00Z


In [4]:
def explode_pauses(row):
    events = json.loads(row['typing_metadata_json']).get('pause_events', [])
    total_pause = sum(item['duration_ms'] for item in events)
    return pd.Series({
        'pause_count': len(events),
        'total_pause_ms': total_pause
    })

features = df.apply(explode_pauses, axis=1) if not df.empty else pd.DataFrame(columns=['pause_count', 'total_pause_ms'])
features.head()

,pause_count,total_pause_ms
0,0,0
1,0,0
